In [ ]:
""" change working directory """
import os

if os.path.basename(os.getcwd()) == 'working':
    os.chdir('../input/cvhomeworkfocalloss')
print('current working directory is {}'.format(os.getcwd()))

print(os.getcwd())
print(os.listdir(os.getcwd()))


In [ ]:
import torch
import torchvision.transforms as transforms
from models.efficientNet.model import EfficientNet
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image
import numpy as np
import pandas as pd
import random
import os
from tqdm.notebook import tqdm
from imgaug import augmenters as iaa


## Cfg

In [ ]:
class cfg:
    """Main config."""
    NUMCLASSES = 5  # CONST
    seed = 42  # random seed

    pathtoimgs = "../cassava-leaf-disease-classification/test_images"  # Path to folder with train images
    # pathtoimgs = "E:/0_Project/2020.03/computer_vision_dataset/cassava-leaf-disease-classification/test_images"
    pathtocsv = "../cassava-leaf-disease-classification/sample_submission.csv"  # Path to csv-file with targets
    # pathtocsv = "./sample_submission.csv"
    chk = "./best2.pth"  # Path to model checkpoint (weights)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Device
    modelname = "efficientnet-b4"  # PyTorch model
    batchsize = 1  # BatchSize
    numworkers = 4  # Number of workers

def fullseed(seed=42):
    """Sets the random seeds."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ['PYTHONHASHSEED'] = str(seed)
fullseed(cfg.seed)

## load model

In [ ]:
def get_model(cfg):
    """Get PyTorch model."""
    model = EfficientNet.from_name(cfg.modelname, in_channels=3, num_classes=5)
    model.load_state_dict(torch.load(cfg.chk))
    return model.to(cfg.device)

## dataset class

In [ ]:
class Single_Dataset(Dataset):
    def __init__(self, cfg, images, is_train , transform = None):
        super(Single_Dataset,self).__init__()
        self.is_train = is_train
        self.transform = transform
        self.cfg = cfg  # Config
        self.images = images  # List with images

        self.seq = iaa.SomeOf((3, 11), {
            # self.seq = iaa.SomeOf((0, 5), {
            # iaa.Fliplr(0.5),
            iaa.Flipud(0.5),
            # iaa.Crop(percent=(0, 0.1)),
            # Small gaussian blur with random sigma between 0 and 0.5.
            # But we only blur about 50% of all images.
            iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
            # Strengthen or weaken the contrast in each image.
            iaa.ContrastNormalization((0.75, 1.5)),
            # 先将图片从RGB变换到HSV,然后将H值增加10,然后再变换回RGB
            iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB",
                               children=iaa.WithChannels(2, iaa.Add((10, 50)))),
            iaa.AverageBlur(k=((2, 5), (1, 3))),
            iaa.SimplexNoiseAlpha(
                first=iaa.EdgeDetect((0.0, 0.2)),
                second=iaa.ContrastNormalization((0.5, 2.0)),
                per_channel=True
            ),
            # Add gaussian noise.
            # For 50% of all images, we sample the noise once per pixel.
            # For the other 50% of all images, we sample the noise per pixel AND
            # channel. This can change the color (not only brightness) of the
            # pixels.
            iaa.ImpulseNoise(p=0.02),
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.02 * 255), per_channel=0.5),
            # Make some images brighter and some darker.
            # In 20% of all cases, we sample the multiplier once per channel,
            # which can end up changing the color of the images.
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            iaa.PerspectiveTransform(scale=0.06),
            # # 图像扭曲
            # iaa.PiecewiseAffine(scale=(0.01, 0.05)),
            # Apply affine transformations to each image.
            # Scale/zoom them, translate/move them, rotate them and shear them.
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-45, 45),
                shear=(-8, 8)
            )
        }, random_order=True)

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.cfg.pathtoimgs, self.images[index])).convert('RGB')
        if self.is_train:
            img = self.seq.augment_image(np.array(img))
            img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.images)


## load data

In [ ]:
def get_loader(cfg):
    """Getting dataloaders for train, validation (and test, if needed)."""
    data = pd.read_csv(cfg.pathtocsv)
    imgs = list(data["image_id"])
    test_transform = transforms.Compose([transforms.Resize((512, 512)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=(0.42984136, 0.49624753, 0.3129598), std=(0.21417203, 0.21910103, 0.19542212))
                                         ])
    test_dataset = Single_Dataset(
        cfg = cfg,
        images = imgs,
        is_train=False,
        transform=test_transform
    )
    dataloader = torch.utils.data.DataLoader(test_dataset,
                                             shuffle=False,
                                             batch_size=cfg.batchsize,
                                             pin_memory=True,
                                             num_workers=cfg.numworkers)
    return dataloader

## Inference

In [ ]:
torch.cuda.empty_cache()
dataloader = get_loader(cfg)
model = get_model(cfg)
model.eval()
preds = []
with torch.no_grad():
    for img in tqdm(dataloader):
        img = Variable(img)
        outputs = model(img.to(cfg.device))
        preds.append(np.argmax(outputs.to('cpu').numpy()).tolist())

In [ ]:
print(preds)

## save result

In [ ]:
# Creating submission file
df = pd.read_csv(cfg.pathtocsv)
df.head()
df["label"] = preds
df.to_csv('/kaggle/working/submission.csv', index=False)
df.head()
